### Dlaczego sekwencje są iterowalne - funkcja `iter`

In [1]:
import re
import reprlib

RE_WORD = re.compile(r"\w+")


class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __getitem__(self, index):
        return self.words[index]

    def __len__(self):
        return len(self.words)

    def __repr__(self):
        return "Sentence({})".format(reprlib.repr(self.text))

In [2]:
s = Sentence("Fugiat ab perspiciatis voluptatem illum ullam.")
s

Sentence('Fugiat ab pe... illum ullam.')

In [3]:
for word in s:
    print(word)

Fugiat
ab
perspiciatis
voluptatem
illum
ullam


In [5]:
s[0], s[1], s[-1]

('Fugiat', 'ab', 'ullam')

Gdy interpreter musi iterować po obiekcie $x$, automatycznie wywołuje `iter(x)`. Wbudowana funkcja `iter`:
 - Sprawdza czy obiekt implementuje metodę `__iter__` i wywołuje ją w celu uzyskania iteratora.
 - Jeśli metoda `__iter__` nie jest zaimplementowana, ale zaimplementowana jest metoda `__getitem__`, Python tworzy iterator, który próbuje pobierać elementy kolejno zaczynając od indeksu 0.
 - Jeśli drugi krok się nie uda, Python wywołuje `TypeError` z informacją, że obiekt nie jest iterowalny.
  
Dlateg właśnie dowolna sekwencja jest iterowalna: ponieważ wszystkie implementują `__getitem__`. Nie mniej standardowe sekwencje implementują `__iter__` i powinniśmy robić to samo we własnych implementacjach.

In [8]:
from collections import abc


class GooseSpam:
    def __iter__(self): ...

# W podejściu gęsiego typowania obiekt jest traktowany jako iterowalny, jeśli 
# implementuje metodę __iter__. Nie potrzebne jest tworzenie klasy podrzędnej
# ani rejestrowanie, ponieważ abc.Iterable implementuje __subclasshook__.
issubclass(GooseSpam, abc.Iterable), isinstance(GooseSpam(), abc.Iterable)

(True, True)

W Pythonie 3.10+ najbardziej dokładną metodą sprawdzenia czy obiekt jest iterowalny, jest wywołanie funkcji `iter` na tym obiekcie i obsłużenie wyjątku `TypeError` jeśli nie jest.

### Używanie `iter` wobec obiektów iterowalnych

In [15]:
import random


def d6():
    return random.randint(1, 6)


d6_iter = iter(d6, 1)
d6_iter

In [16]:
for roll in d6_iter:
    print(roll, end=" ")

3 5 3 4 5 4 

Funkcję `iter` można wywołać z dwoma argumentami, aby utworzyć iterator z funkcji lub dowolnego wywoływalnego obiektu. W takim przypadku pierwszy argument musi być obiektem wywoływalnym, który będzie wywoływany wielokrotnie bez żadnego argumentu w celu tworzenia wartości. Drugi argument to wartownik (sentinel). Jeżeli wartość wartownika zostanie zwrócona przez obiekt wywoływalny, to nastąpi zgłoszenie wyjątki `StopIteration` zamiast zwracania wartownika.

### Obiekty iterowalne kontra iteratory

Obiekt iterowalny to dowolny obiekt, z którego wbudowana funkcja `iter` może uzyskać iterator. Obiekty implementujące metodę `__iter__` zwracającą iterator, są iterowalne. 

Python uzyskuje iteratory z obiektów iterowalnych.

In [18]:
# Gdyby nie było pętli for to musielibyśmy pisać coś takiego:
s = "ABC"
it = iter(s)

while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break


A
B
C


Standardowy interfejs dla iteratora ma dwie metody:
 - `__next__` - Zwraca następny dostępny element zgłaszając `StopIteration`, gdy nie ma więcej elementów.
 - `__iter__` - Zwraca `self`. Pozwala to na używanie iteratorów tam, gdzie oczekiwany jest obiekt iterowalny.

### Klasa Sentence - klasyczny wzorzec iteratora

In [19]:
RE_WORD = re.compile(r"\w+")


class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return "Sentence({})".format(reprlib.repr(self.text))

    # Nie ma metody __getitem__ aby było jasne, że klasa jest iterowalna
    # ponieważ implementuje metodę __iter__
    def __iter__(self):
        return SentenceIterator(self.words)


class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0

    # Iteratory powinny implementować metodę __next__ 
    # i zwracać self w metodzie __iter__
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

In [20]:
c = Sentence("Fugiat ab perspiciatis voluptatem illum ullam.")
for w in c:
    print(w)

Fugiat
ab
perspiciatis
voluptatem
illum
ullam


**NIE PRZEKSZTAŁCAJ OBIEKTU ITEROWALNEGO W SWÓJ WŁASNY ITERATOR**

Obiekty iterowalne mają metodę `__iter__`, która za każdym razem tworzy nowe wystąpienie iteratora. Iteratory implementują metodę `__next__`, która zwraca pojedyncze elementy oraz metodę `__iter__`, która zwraca `self`.

**Iteratory są też iterowalne, ale obiekty iterowalne nie są iteratorami.** Obiekty iterowalne muszą implementować `__iter__` ale nie powinny implementować `__next__`.

### Klasa Sentence - funkcja generatora

In [22]:
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return "Sentence({})".format(reprlib.repr(self.text))

    # Funkcja generatora nie zgłasza StopIteration. Po prostu kończy działanie
    # gdy wyprodukuje wszystkie wartości.
    def __iter__(self):
        for word in self.words:
            yield word

In [23]:
s = Sentence("Fugiat ab perspiciatis voluptatem illum ullam.")
for w in s:
    print(w)

Fugiat
ab
perspiciatis
voluptatem
illum
ullam


Każda funkcja Pythona, która zawiera słowo kluczowe `yield`, jest funkcją generatora. Funkcją która w momencie wywołania zwraca obiekt generatora. Funkcja generatora jest fabryką generatorów.

In [24]:
def gen123():
    yield 1
    yield 2
    yield 3


In [25]:
gen123

<function __main__.gen123()>

In [26]:
gen123()

<generator object gen123 at 0x000001CD1A8A7C10>

In [27]:
g = gen123()
next(g), next(g), next(g)

(1, 2, 3)

In [28]:
next(g)

StopIteration: 

Generatory są iteratorami, które produkują wartości z wyrażeń przekazywanych do instrukcji `yield`.

In [29]:
def genAB():
    print("start")
    yield "A"
    print("continue")
    yield "B"
    print("end.")
    
for c in genAB():
    print("-->", c)

start
--> A
continue
--> B
end.


### Leniwa klasa Sentence

In [33]:
RE_WORD = re.compile(r"\w+")


class Sentence:
    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return "Sentence({})".format(reprlib.repr(self.text))

    def __iter__(self):
        # Wyrażenie generatora.
        return (match.group() for match in RE_WORD.finditer(self.text))

### Iterator vs Generator

- Iterator - ogólny termin dla dowolnego obiektu, który implementuje `__next__`. Iteratory są zaprojektowane dla tworzenia danych, które są konsumowane przez kod kliencki, czyli kod, który steruje iteratorem poprzez pętlę `for` albo inną funkcjonalność iteracyjną. W praktyce większość iteratorów, których używamy to generatory.
- Generator - iterator budowany przez kompilator Pythona. Aby utworzyc generator, nie implementujemy `__next__`. Zamiast tego używamy słowa kluczowego `yield` aby utworzyć funkcję generatora, która jest fabryką obiektów generatorów. Wyrażenie generatora jest innym sposobem zbudowania obiektu generatora. Obiekty generatora udostępniają metodę `__next__`, zatem są one iteratorami.

### Generator ciągu arytmetycznego

In [47]:
class ArithmeticProgression:
    def __init__(self, start, step, stop=None):
        self.start = start
        self.step = step
        self.stop = stop

    def __iter__(self):
        # Wartości start i step mogą mieć różne typy, więc wynik powinien
        # być zgodny z typem start + step.
        result_type = type(self.start + self.step)
        result = result_type(self.start)
        index = 0
        while self.stop is None or result < self.stop:
            yield result
            index += 1
            # Zapobiegaj błędom zaokrągleń.
            result = self.start + self.step * index

In [48]:
[v for v in ArithmeticProgression(0, 1, 3)]

[0, 1, 2]

In [49]:
[v for v in ArithmeticProgression(1, .5, 3)]

[1.0, 1.5, 2.0, 2.5]

In [51]:
from fractions import Fraction

[v for v in ArithmeticProgression(0, Fraction("1/3"), 1)]

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [55]:
from decimal import Decimal

[v for v in ArithmeticProgression(0, Decimal("0.09"), 0.3)]

[Decimal('0'), Decimal('0.09'), Decimal('0.18'), Decimal('0.27')]

Jeżeli jedynym zadaniem klasy jest budowanie generatora przez implementację `__iter__`, to klasa może być zredukowana do funkcji generatora. Funkcja generatora jest przecież fabryką generatorów.

In [58]:
def arithmetic_progession(start, step, stop=None):
    result = type(start + step)(start)
    index = 0
    while stop is None or result < stop:
        yield result
        index += 1
        result = start + step * index

In [59]:
[v for v in arithmetic_progession(0, Decimal("0.1"), 0.3)]

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

### Funkcje generatora w bibliotece standardowej

In [40]:
import functools
import itertools
import operator

#### 1. Filtrujące funkcje generatora:

In [24]:
def vowel(c):
    return c.lower() in "aeiou"

In [28]:
# filter(predicate, it)
# Zwraca tylko te elementy z it, dla których predicate zwraca True.
list(filter(vowel, "Aardvark"))

['A', 'a', 'a']

In [29]:
# filterfalse(predicate, it)
# Zwraca tylko te elementy z it, dla których predicate zwraca False.
list(itertools.filterfalse(vowel, "Aardvark"))

['r', 'd', 'v', 'r', 'k']

In [30]:
# islice(it, start, stop, step=1)
# Zwraca wycinek it zaczynający się od start, kończący na stop, z krokiem step.
list(itertools.islice("Aardvark", 4))

['A', 'a', 'r', 'd']

In [31]:
list(itertools.islice("Aardvark", 4, 7))

['v', 'a', 'r']

In [32]:
list(itertools.islice("Aardvark", 1, 7, 2))

['a', 'd', 'a']

In [33]:
# compress(it, selector_it)
# Zwraca tylko te elementy z it, dla których odpowiadające im elementy 
# z selector_it są True.
list(itertools.compress("Aardvark", (1, 0, 1, 1, 0, 1, 1)))

['A', 'r', 'd', 'a', 'r']

In [34]:
# dropwhile(predicate, it)
# Pomija elementy z it, dopóki predicate zwraca True, a następnie zwraca
# wszystkie pozostałe elementy.
list(itertools.dropwhile(vowel, "Aardvark"))

['r', 'd', 'v', 'a', 'r', 'k']

#### 2. Mapujące funkcje generatora:

In [36]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]

In [37]:
# accumulate(it, [func])
# Zwraca zakumulowaną sumę elementów it. Jeśli podano func, to zwraca
# zakumulowane wyniki wywołań func na elementach it. 
list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [38]:
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [39]:
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [42]:
list(itertools.accumulate(sample, operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [44]:
# enumerate(iterable, start=0)
# Produkuje krotki (index, value) dla elementów z iterable.
# gdzie index zaczyna się od start.
list(enumerate("abc", 1))

[(1, 'a'), (2, 'b'), (3, 'c')]

In [47]:
# map(func, it1, [it2, ...])
# Stosuje func wobec każdego elementu z it. Jeśli podano więcej niż jeden
# iterowalny, to func musi przyjmować tyle argumentów, ile jest iterowalnych.
list(map(operator.mul, range(11), range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [50]:
list(map(lambda a, b: (a, b), range(11), [2, 4, 8]))  # To właśnie robi zip

[(0, 2), (1, 4), (2, 8)]

In [54]:
# starmap(func, it)
# Stosuje func wobec każdego elementu z it. Wejściowy obiekt iterowalny
# powinien produkować elementy iterowalne iit a func jest stosowana jako
# func(*iit).
list(itertools.starmap(operator.mul, enumerate("abc", 1)))

['a', 'bb', 'ccc']

In [55]:
# Średnia krocząca.
list(itertools.starmap(lambda a, b: b / a, enumerate(itertools.accumulate(sample), 1)))

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

#### 3. Scalające funkcje generatora:

In [56]:
# chain(it1, it2, ...)
# Łączy iterowalne w jedno.
list(itertools.chain("ABC", range(2)))

['A', 'B', 'C', 0, 1]

In [57]:
# chain.from_iterable(it)
# Produkuje elementy z każdego elementu it, który powinien być iterowalny.
list(itertools.chain.from_iterable(enumerate("ABC")))

[0, 'A', 1, 'B', 2, 'C']

In [58]:
# product(it1, it2, ..., repeat=1)
# Iloczyn kartezjański.
list(itertools.product("ABC", range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [59]:
# zip(it1, it2, ...)
# Produkuje krotki z elementów z it1, it2, ...
list(zip("ABC", range(5)))

[('A', 0), ('B', 1), ('C', 2)]

In [61]:
# zip_longest(it1, it2, ..., fillvalue=None)
# Produkuje krotki z elementów z it1, it2, ... Wypełnia brakujące wartości
list(itertools.zip_longest("ABC", range(5), fillvalue="?"))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

#### 4. Rozszerzające funkcje generatora:

In [65]:
# combinations(it, out_len)
# Zwraca kombinacje out_len elementowe elementów z it.
list(itertools.combinations("ABCD", 3))

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'C', 'D'), ('B', 'C', 'D')]

In [66]:
# combinations_with_replacement(it, out_len)
# Zwraca kombinacje out_len elementowe elementów z it z powtórzeniami.
list(itertools.combinations_with_replacement("ABC", 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [69]:
# count(start=0, step=1)
# Produkuje liczby zaczynając od start, z krokiem step bez końca.
list(itertools.islice(itertools.count(1, .3), 3))

[1, 1.3, 1.6]

In [70]:
# cycle(it)
# Produkuje elementy z it w nieskończoność.
list(itertools.islice(itertools.cycle("ABC"), 7))

['A', 'B', 'C', 'A', 'B', 'C', 'A']

In [71]:
# permutations(it, out_len=None)
# Zwraca permutacje out_len elementowe elementów z it.
list(itertools.permutations("ABC", 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [72]:
# repeat(elem, [times])
# Produkuje elem times razy lub w nieskończoność.
list(itertools.repeat(8, 4))

[8, 8, 8, 8]

In [75]:
# pairwise(it)
# Produkuje pary kolejnych elementów z it.
list(itertools.pairwise("ABC"))

[('A', 'B'), ('B', 'C')]

#### 5. Przestawiające funkcje generatora:

In [80]:
# groupby(it, key=None)
# Produkuje 2-krotki, grupując elementy z it według wartości zwracanej przez key.
list(list(group) for char, group in itertools.groupby("LLLLAAGGG"))

[['L', 'L', 'L', 'L'], ['A', 'A'], ['G', 'G', 'G']]

In [81]:
# reversed(seq)
# Produkuje elementy z seq w odwrotnej kolejności.
# Jednak unika tworzenia kopii seq.
list(reversed("ABC"))

['C', 'B', 'A']

In [87]:
# tee(it, n=2)
# Produkuje krotkę złożoną z n generatorów, każdy z nich produkuje
# elementy z it.
g1, g2, g3 = itertools.tee("ABC", 3)
list(g1), list(g2), list(g3)

(['A', 'B', 'C'], ['A', 'B', 'C'], ['A', 'B', 'C'])

In [88]:
list(zip(*itertools.tee("ABC", 3)))

[('A', 'A', 'A'), ('B', 'B', 'B'), ('C', 'C', 'C')]

#### 6. Funkcje redukujące

In [89]:
# all(it)
# Zwraca True, jeśli wszystkie elementy z it są prawdziwe.
# all([]) zwraca True.
all([1, 2, 3])

True

In [90]:
# any(it)
# Zwraca True, jeśli którykolwiek element z it jest prawdziwy.
# any([]) zwraca False.
any([1, 2, 3])

True

In [91]:
# max(it, [key], [default])
# Zwraca największy element z it. Klucz jest funkcją porządkującą.
max(("ab", "bc", "a", "abc"), key=len)

'abc'

In [92]:
# min(it, [key], [default])
# Zwraca najmniejszy element z it. Klucz jest funkcją porządkującą.
min(("ab", "bc", "a", "abc"), key=len)

'a'

In [106]:
# reduce(func, it, [initial])
# Zwraca wynik wywołań func na elementach it. Jeśli podano initial,
# to jest on używany jako pierwszy argument func.
functools.reduce(operator.truediv, range(1, 5), 2)

0.08333333333333333

In [99]:
# sum(it, start=0)
# Zwraca sumę elementów z it. Jeśli podano start, to jest on dodawany do sumy.
sum(range(10), 100)

145

### Podgeneratory wykorzystujące `yield from`

In [313]:
def sub_gen():
    yield 1.1
    yield 1.2


def gen():
    yield 1
    yield from sub_gen()
    yield 2


for x in gen():
    print(x)

1
1.1
1.2
2


Składnia wyrażenia `yield from` została wprowadzona w Pythonie 3.3, aby umożliwić delegowanie pracy przez generator do podgeneratora. Składnia `yield from` wstrzymuje działanie generatora `gen()` i wartości są produkowane z `sub_gen()` aż do wyczerpania. Następnie produkowane są wartości z `gen()`. Gdy podgenerator zawiera inttrukcję `return` z jakąś wartością, to może zostać ona przechwycona przez delegujący generator poprzez użycie `yield from` jako części wyrażenia.

In [314]:
def sub_gen():
    yield 1.1
    yield 1.2
    return "Done!"


def gen():
    yield 1
    result = yield from sub_gen()
    print(result)
    yield 2


for x in gen():
    print(x)

1
1.1
1.2
Done!
2


In [317]:
# Ponowne wynalezienie generatora chain.
def my_chain(*iterables):
    for it in iterables:
        yield from it

In [318]:
list(my_chain("abc", range(2)))

['a', 'b', 'c', 0, 1]

### Przechodzenie przez drzewo

In [323]:
def tree(cls, level=0):
    yield cls.__name__, level
    for sub_cls in cls.__subclasses__():
        yield from tree(sub_cls, level + 1)


def display(cls):
    for name, level in tree(cls):
        print(f"{'-' * 4 * level}{name}")
        
display(BaseException)

BaseException
----BaseExceptionGroup
--------ExceptionGroup
----Exception
--------ArithmeticError
------------FloatingPointError
------------OverflowError
------------ZeroDivisionError
----------------DivisionByZero
----------------DivisionUndefined
------------DecimalException
----------------Clamped
----------------Rounded
--------------------Underflow
--------------------Overflow
----------------Inexact
--------------------Underflow
--------------------Overflow
----------------Subnormal
--------------------Underflow
----------------DivisionByZero
----------------FloatOperation
----------------InvalidOperation
--------------------ConversionSyntax
--------------------DivisionImpossible
--------------------DivisionUndefined
--------------------InvalidContext
--------AssertionError
--------AttributeError
------------FrozenInstanceError
--------BufferError
--------EOFError
------------IncompleteReadError
--------ImportError
------------ModuleNotFoundError
----------------PackageNotFoundE